In [1]:
#temporary cell - for testing; after incorporating into the main code, it will not be used
#Agent class will use Persona class and will supply ai client object to it

!pip install openai
from openai import OpenAI
from google.colab import userdata
secret_key = userdata.get('openai_key')
client = OpenAI(api_key=secret_key)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


In [4]:


class Persona:
    """
    Class Persona constructs a persona based on a given raw string input with specific model.

    Attributes:
    raw_string : str
        A raw string input defining persona attributes
    model : str
        An AI model used to process the persona attributes
    persona_desc : str
        Description of persona obtained from raw string, processed by an AI model
    """
    def __init__(self, aiclient,raw_string, model_name):
        self.raw_string = raw_string
        self.model = model_name
        self.aiclient = aiclient
        if raw_string:
            self.persona_desc = self.persona_from_rawstring(raw_string, model_name)

    def persona_from_rawstring(self, raw_string, modelname):
        """
        Method to retrieve and format the persona description from the raw string input provided using OpenAI's language model.

        Parameters:
        raw_string : str
            string input that describes the persona
        modelname : str
            name of AI model used for processing

        Returns:
        str
            A formatted persona_description
        """
        demographic = []
        questions = self.get_questions(demographic)
        prompt = f" {raw_string} \nExtract information from the text provided, do not use any outside information. What is the person's: \n{questions} "

        response = self.aiclient.chat.completions.create(
            model=modelname,
            response_format={"type": "json_object"},
            temperature=0,
            messages=[
                {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
                {"role": "user", "content": f" {prompt} "}
            ]
        )

        result = response.choices[0].message.content
        return self.format_persona(result, demographic)

    def get_questions(self, arr_info):
        """
        Method to retrieve demographic questions from file.

        Parameters:
        arr_info : list
            An array used to store info from the file

        Returns:
        str
            A string of questions retrieved from the file
        """

        with open('demographic_questions.txt', 'r') as file:
            lines = file.readlines()
        questions = ""

        for line in lines:
            line = line.strip()
            parts = line.split('^')
            questions += f"^ {parts[0]}? {parts[1]} Do not answer this question if not mentioned , do not assume\n"
            arr_info.append(parts)
        return questions

    def format_persona(self, demographic_info_json, arrdemographic):

        import json
        demographic_info = json.loads(demographic_info_json)

        persona_desc = ''

        for key, value in demographic_info.items():
            for i in range(len(arrdemographic)):
                if arrdemographic[i][0].strip() == key:
                    if value and value not in ['not provided', 'not mentioned']:
                        persona_desc += arrdemographic[i][2] + " " + str(value) + ". "
                    break
        return persona_desc


raw_str = "a 20 year old man from Ohio who is an independent with a graduate degree"

persona1 = Persona(client,raw_str, "gpt-3.5-turbo-0125")
persona2 = Persona(client, raw_str, "gpt-4-1106-preview")

print('gpt-3.5-turbo-0125:', persona1.persona_desc)
print('gpt-4-1106-preview:', persona2.persona_desc)


gpt-3.5-turbo-0125: your age is 20. your gender is male. your education is graduate degree. politically, you identify as a independent. you live in Ohio. 
gpt-4-1106-preview: your age is 20. your gender is male. your education is graduate degree. politically, you identify as a independent. you live in Ohio. 
